# Computation of Scalar Products

By now, we are able to construct orthogonal polynomials, and to construct quadrature rules for a given nonnegative weight function, respectively.
Now we combine both ideas to solve integrals involving the orthogonal polynomials
$$
\langle \phi_{i_1} \phi_{i_2} \cdots \phi_{i_{m-1}}, \phi_{i_m} \rangle
= \int \phi_{i_1}(t) \phi_{i_2}(t) \cdots \phi_{i_{m-1}}(t) \phi_{i_m}(t) w(t) \mathrm{d} t,
$$
both for the univariate and multivariate case.
The integrand is a polynomial (possibly multivariate) that can be solved exactly with the appropriate Gauss quadrature rules.

Notice: To simplify notation we drop the integration interval.
It is clear from the context.


## Univariate Polynomials
### Classical Polynomials
Let's begin with a univariate basis for some *classical* orthogonal polynomial

In [121]:
using PolyChaos
deg, n = 4, 20
s_α, s_β = 2.1, 3.2
op = OrthoPoly("beta01",deg,Dict(:shape_a=>s_α,:shape_b=>s_β);Nrec=n)


Univariate orthogonal polynomials
name:		beta01
degree:		4
#coeffs:	20
α =		[0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143]...
β =		[1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388]...

Measure dλ(t)=w(t)dt
name:		beta01
w(t):		getfield(PolyChaos, Symbol("##65#66")){Float64,Float64}(2.1, 3.2)
dom:		(0.0, 1.0)
symmetric:	false
pars:		:shape_b => 3.2
		:shape_a => 2.1


To add the corresponding quadrature rule there is the composite struct `OrthoPolyQ` whose simplest constructor reads

In [122]:
opq = OrthoPolyQ(op,n)


Univariate orthogonal polynomials
name:		beta01
degree:		4
#coeffs:	20
α =		[0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143]...
β =		[1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388]...

Measure dλ(t)=w(t)dt
name:		beta01
w(t):		getfield(PolyChaos, Symbol("##65#66")){Float64,Float64}(2.1, 3.2)
dom:		(0.0, 1.0)
symmetric:	false
pars:		:shape_b => 3.2
		:shape_a => 2.1

Quadrature rule
name:		beta01
N:		20
nodes		[0.00801337, 0.0259548, 0.0534374, 0.0899073, 0.134632, 0.186713, 0.245104]...
weights		[0.000950472, 0.00581637, 0.017016, 0.035118, 0.0584312, 0.083353, 0.10533]...


By default, an $n$-point Gauss quadrature rule is create relative to the underlying measure `op.meas`, where $n$ is the number of recurrence coefficients stored in `op.α` and `op.β`.
The type `OrthoPolyQ` has just two fields: an `OrthoPoly`, and a `Quad`.

To compute the squared norms
$$
\| \phi_k \|^2 = \langle \phi_k, \phi_k  \rangle
= \int \phi_k(t) \phi_k(t) w(t) \mathrm{d} t
$$

of the basis we call `computeSP2()`

In [123]:
normsq = computeSP2(opq)

5-element Array{Float64,1}:
 1.0                   
 0.03797318144060756   
 0.0018807430768498865 
 0.00010249372143217376
 5.849823409553846e-6  

For the general case
$$
\langle \phi_{i_1} \phi_{i_2} \cdots \phi_{i_{m-1}}, \phi_{i_m} \rangle
= \int \phi_{i_1}(t) \phi_{i_2}(t) \cdots \phi_{i_{m-1}}(t) \phi_{i_m}(t) w(t) \mathrm{d} t,
$$
there exists a type `Tensor` that requires only two arguments: the *dimension* $m \geq 1$, and an `OrthoPolyQ`

In [124]:
m = 3
t = Tensor(3,opq)


3-dimensional tensor
dim:		3
nonzeros:	25

Univariate orthogonal polynomials
name:		beta01
degree:		4
#coeffs:	20
α =		[0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143]...
β =		[1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388]...

Measure dλ(t)=w(t)dt
name:		beta01
w(t):		getfield(PolyChaos, Symbol("##65#66")){Float64,Float64}(2.1, 3.2)
dom:		(0.0, 1.0)
symmetric:	false
pars:		:shape_b => 3.2
		:shape_a => 2.1

Quadrature rule
name:		beta01
N:		20
nodes		[0.00801337, 0.0259548, 0.0534374, 0.0899073, 0.134632, 0.186713, 0.245104]...
weights		[0.000950472, 0.00581637, 0.017016, 0.035118, 0.0584312, 0.083353, 0.10533]...


To get the desired entries, `Tensor`comes with a `get()` function that is called for some index $a \in \mathbb{N}_0^m$ that has the entries $a = [i_1, i_2, \dots, i_m]$.
For example


In [125]:
t.get([1,2,3])

0.00010249372143217382

Or using comprehension

In [126]:
T = [ t.get([i1,i2,i3]) for i1=0:dim(opq)-1,i2=0:dim(opq)-1,i3=0:dim(opq)-1]

5×5×5 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0        0.0         0.0          0.0       
 0.0  0.0379732  0.0         0.0          0.0       
 0.0  0.0        0.00188074  0.0          0.0       
 0.0  0.0        0.0         0.000102494  0.0       
 0.0  0.0        0.0         0.0          5.84982e-6

[:, :, 2] =
 0.0         0.0379732     0.0           0.0           0.0        
 0.0379732   0.00215924    0.00188074   -2.14723e-19   5.05573e-20
 0.0         0.00188074    0.000144891   0.000102494  -1.92435e-20
 0.0        -2.14723e-19   0.000102494   8.86598e-6    5.84982e-6 
 0.0         5.05573e-20  -1.92435e-20   5.84982e-6    5.36411e-7 

[:, :, 3] =
 0.0          0.0          0.00188074   0.0           0.0        
 0.0          0.00188074   0.000144891  0.000102494  -1.92435e-20
 0.00188074   0.000144891  0.000127149  1.0934e-5     5.84982e-6 
 0.0          0.000102494  1.0934e-5    7.80614e-6    7.09802e-7 
 0.0         -1.92435e-20  5.84982e-6   7.09802e-7    4.73123e-7 

[:, :, 

Notice that we can cross-check the results.

In [127]:
using LinearAlgebra
@show normsq == LinearAlgebra.diag(T[:,:,1])
@show normsq == LinearAlgebra.diag(T[:,1,:])
@show normsq == LinearAlgebra.diag(T[1,:,:])

normsq == LinearAlgebra.diag(T[:, :, 1]) = true
normsq == LinearAlgebra.diag(T[:, 1, :]) = true
normsq == LinearAlgebra.diag(T[1, :, :]) = true


true

Also, `normsq` can be computed analogously in `Tensor` format

In [128]:
t2 = Tensor(2,opq)
@show normsq == [ t2.get([i,i]) for i=0:dim(opq)-1]

normsq == [t2.get([i, i]) for i = 0:dim(opq) - 1] = true


true

### Arbitrary Weights
Of course, the type `OrthoPolyQ` can be constructed for arbitrary weights $w(t)$.
In this case we have to compute the orthogonal basis and the respective quadrature rule.
Let's re-work the above example by hand.

In [129]:
using SpecialFunctions
supp = (0,1)
function w(t)
    supp[1]<=t<=supp[2] ? (t^(s_α-1)*(1-t)^(s_β-1)/SpecialFunctions.beta(s_α,s_β)) : error("$t not in support")
end
my_meas = Measure("my_meas",w,supp,false,Dict())
my_op = OrthoPoly("my_op",deg,my_meas;Nrec=n)
my_quad = Quad(n,my_op)
my_opq = OrthoPolyQ(my_op,my_quad)


Univariate orthogonal polynomials
name:		my_op
degree:		4
#coeffs:	20
α =		[0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143]...
β =		[1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388]...

Measure dλ(t)=w(t)dt
name:		my_meas
w(t):		w
dom:		(0.0, 1.0)
symmetric:	false

Quadrature rule
name:		golubwelsch
N:		20
nodes		[0.00801337, 0.0259548, 0.0534374, 0.0899073, 0.134632, 0.186713, 0.245104]...
weights		[0.000950472, 0.00581637, 0.017016, 0.035118, 0.0584312, 0.083353, 0.10533]...


Now we can compute the squared norms $\| \phi_k \|^2$

In [130]:
my_normsq = computeSP2(my_opq)

5-element Array{Float64,1}:
 0.9999999999996537   
 0.037973181440551666 
 0.0018807430768424916
 0.0001024937214313095
 5.849823409460972e-6 

And the tensor

In [131]:
my_t = Tensor(m,my_opq)
my_T = [ my_t.get([i1,i2,i3]) for i1=0:dim(opq)-1,i2=0:dim(opq)-1,i3=0:dim(opq)-1]

5×5×5 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0        0.0         0.0          0.0       
 0.0  0.0379732  0.0         0.0          0.0       
 0.0  0.0        0.00188074  0.0          0.0       
 0.0  0.0        0.0         0.000102494  0.0       
 0.0  0.0        0.0         0.0          5.84982e-6

[:, :, 2] =
 0.0         0.0379732     0.0           0.0           0.0        
 0.0379732   0.00215924    0.00188074   -1.54774e-18  -5.26219e-19
 0.0         0.00188074    0.000144891   0.000102494  -1.71842e-19
 0.0        -1.54774e-18   0.000102494   8.86598e-6    5.84982e-6 
 0.0        -5.26219e-19  -1.71842e-19   5.84982e-6    5.36411e-7 

[:, :, 3] =
 0.0          0.0          0.00188074   0.0           0.0        
 0.0          0.00188074   0.000144891  0.000102494  -1.71842e-19
 0.00188074   0.000144891  0.000127149  1.0934e-5     5.84982e-6 
 0.0          0.000102494  1.0934e-5    7.80614e-6    7.09802e-7 
 0.0         -1.71842e-19  5.84982e-6   7.09802e-7    4.73123e-7 

[:, :, 

Let's compare the results:

In [132]:
@show abs.(normsq-my_normsq)
@show norm(T-my_T)

abs.(normsq - my_normsq) = [3.46279e-13, 5.58928e-14, 7.39491e-15, 8.64258e-16, 9.28738e-17]
norm(T - my_T) = 3.6007147263160944e-13


3.6007147263160944e-13

Notice: __The possibility to create quadrature rules for arbitrary weights should be reserved to cases different from *classical* ones.__

### Multivariate Polynomials
For multivariate polynomials the syntax for `Tensor` is very much alike, except that we are dealing with the type `MultiOrthoPoly` now.

In [133]:
mop = MultiOrthoPoly([opq,my_opq],deg)


2-variate orthogonal polynomials
name:		beta01
		my_op
deg:		4
dim:		15
ind:		[0, 0]
		[1, 0]
		[0, 1]
		[2, 0]
		[1, 1]
		[0, 2]
		[3, 0]
		...



In [134]:
mt2 = Tensor(2,mop)
mt3 = Tensor(3,mop)
mT2 = [ mt2.get([i,i]) for i=0:dim(mop)-1 ]

15-element Array{Float64,1}:
 0.9999999999996537    
 0.03797318144059441   
 0.037973181440551666  
 0.0018807430768492354 
 0.00144196250871918   
 0.0018807430768424916 
 0.00010249372143213827
 7.141779810028215e-5  
 7.141779810010645e-5  
 0.0001024937214313095 
 5.849823409551821e-6  
 3.892012680461293e-6  
 3.5371945211048704e-6 
 3.892012680434203e-6  
 5.849823409460972e-6  

Notice that `mT2` carries the elements of the 2-dimensional tensors for the univariate bases `opq` and `my_opq`.
The encoding is given by the multi-index `mop.ind`

In [135]:
mop.ind

15×2 Array{Int64,2}:
 0  0
 1  0
 0  1
 2  0
 1  1
 0  2
 3  0
 2  1
 1  2
 0  3
 4  0
 3  1
 2  2
 1  3
 0  4

To cross-check the results we can distribute the multi-index back to its univariate indices with the help of `findUnivariateIndices`.

In [139]:
ind_opq = findUnivariateIndices(1,mop.ind)
ind_my_opq = findUnivariateIndices(2,mop.ind)

5-element Array{Int64,1}:
  1
  3
  6
 10
 15

In [140]:
@show mT2[ind_opq] - normsq
@show mT2[ind_my_opq] - my_normsq;

mT2[ind_opq] - normsq = [-3.46279e-13, -1.31492e-14, -6.51172e-16, -3.54941e-17, -2.02526e-18]
mT2[ind_my_opq] - my_normsq = [0.0, 0.0, 0.0, 0.0, 0.0]
